In [1]:
import calendar
import datetime
import importlib 
import numpy as np
import os

from parcels import Field, FieldSet, ParticleSet,Variable, JITParticle

In [2]:
import OP_functions_shared as OP
import PBDEs_OP_Kernels_shared as PBDE
import Susans_Model_Driver as Susan

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
importlib.reload(PBDE)
importlib.reload(OP)
importlib.reload(Susan)

<module 'Susans_Model_Driver' from '/ocean/sallen/allen/research/Meopar/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/Susans_Model_Driver.py'>

In [ ]:
year, month = 2021, 1
sim_length = 500 # days
number_outputs = 100
Susan.PBDEs_OP_run(year, month, sim_length, number_outputs)

2976


In [138]:
year, month = 2021, 1
sim_length = 200 # days

(start_time, data_length, duration, delta_t, 
 release_particles_every, number_particles) = Susan.timings(year, month, sim_length)


2976


In [ ]:
field_set, constants = Susan.set_fieldsets_and_constants(start_time, data_length, delta_t)

In [ ]:
path = {'out': '/home/sallen/MEOPAR/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/',}
fn = 'particles_out.zarr'
outfile_states = os.path.join(path['out'], fn)

### Particle Class ###

In [ ]:
class MPParticle(JITParticle):
    # Status Variable #
    status = Variable('status', initial=(np.random.rand(number_particles) > constants['fraction_colloidal']).astype(int) - 2)
    vvl_factor = Variable('fact', initial=1)
    release_time = Variable('release_time', 
                            initial=np.arange(0, release_particles_every*number_particles, release_particles_every))

In [ ]:
pset_states = ParticleSet(field_set, pclass=MPParticle, lon=constants['Iona_clon']*np.ones(number_particles), 
                          depth=constants['Iona_z']*np.ones(number_particles), lat = constants['Iona_clat']*np.ones(number_particles))

In [ ]:
output_interval = datetime.timedelta(seconds=sim_length*84600/100)
output_file = pset_states.ParticleFile(name=outfile_states, outputdt=output_interval)
KE = (pset_states.Kernel(PBDE.PBDEs_states) + pset_states.Kernel(PBDE.Sinking) 
      + pset_states.Kernel(PBDE.Advection)
      + pset_states.Kernel(PBDE.turb_mix) + pset_states.Kernel(PBDE.resuspension)
     )
# Currently have ab/de-sorption, sinking and vertical mixing and resuspension for 5 particles for a bit over an hour.

In [ ]:
#KE = PBDE.Sinking #+ pset.Kernel(tu_mix) #if you want to add more kernels use:  KE = tu_mix + NAME_KERNEL1 + ...
pset_states.execute(KE, runtime=duration, dt=delta_t, output_file=output_file,
                    )